# Install necessary libraries

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 openai==0.28.1

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/

# Define Hyperparameters

In [2]:
model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
dataset_name = "amadeus_TRL_CONCEPTUAL_DATASET_v1.jsonl"
new_model = "llama-2-7b-custom-amadeus"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 5e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

#Load Datasets and Train

In [3]:
# Load datasets
train_dataset = load_dataset('json', data_files='https://raw.githubusercontent.com/Beta-sebas/dataset_1/2db51e1bbb1523296b07302774d4c3d180c411bf/amadeus-train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='https://raw.githubusercontent.com/Beta-sebas/dataset_1/2db51e1bbb1523296b07302774d4c3d180c411bf/amadeus-validation.jsonl', split="train")

print(train_dataset)
print(valid_dataset)

system_message = "Este es un asistente especializado en el modelo de Niveles de Madurez Tecnológica (TRL) de la NASA, capaz de evaluar y clasificar tecnologías según los 9 niveles de madurez. Debes también responder consultas sobre el TRL, abarcando definiciones, actividades recomendadas y criterios de progresión, actuando como una fuente de conocimiento sobre cómo avanzar tecnologías desde la concepción hasta la comercialización."

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['prompt', 'response'],
    num_rows: 265
})
Dataset({
    features: ['prompt', 'response'],
    num_rows: 29
})


In [ ]:
# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)


In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 2.3704, 'learning_rate': 0.0005, 'epoch': 0.07}
{'eval_loss': 1.697920322418213, 'eval_runtime': 6.9481, 'eval_samples_per_second': 4.174, 'eval_steps_per_second': 0.576, 'epoch': 0.07}
{'loss': 1.3482, 'learning_rate': 0.0005, 'epoch': 0.15}
{'eval_loss': 0.6933853030204773, 'eval_runtime': 6.5261, 'eval_samples_per_second': 4.444, 'eval_steps_per_second': 0.613, 'epoch': 0.15}
{'loss': 0.6581, 'learning_rate': 0.0005, 'epoch': 0.22}
{'eval_loss': 0.43921053409576416, 'eval_runtime': 6.4521, 'eval_samples_per_second': 4.495, 'eval_steps_per_second': 0.62, 'epoch': 0.22}
{'loss': 0.7616, 'learning_rate': 0.0005, 'epoch': 0.3}
{'eval_loss': 0.37639087438583374, 'eval_runtime': 6.5803, 'eval_samples_per_second': 4.407, 'eval_steps_per_second': 0.608, 'epoch': 0.3}
{'loss': 0.4784, 'learning_rate': 0.0005, 'epoch': 0.37}
{'eval_loss': 0.2888857126235962, 'eval_runtime': 6.6281, 'eval_samples_per_second': 4.375, 'eval_steps_per_second': 0.603, 'epoch': 0.37}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.3705, 'learning_rate': 0.0005, 'epoch': 0.45}
{'eval_loss': 0.2621997892856598, 'eval_runtime': 6.6675, 'eval_samples_per_second': 4.349, 'eval_steps_per_second': 0.6, 'epoch': 0.45}
{'loss': 0.4805, 'learning_rate': 0.0005, 'epoch': 0.52}
{'eval_loss': 0.25540220737457275, 'eval_runtime': 6.6726, 'eval_samples_per_second': 4.346, 'eval_steps_per_second': 0.599, 'epoch': 0.52}
{'loss': 0.3745, 'learning_rate': 0.0005, 'epoch': 0.6}
{'eval_loss': 0.23537832498550415, 'eval_runtime': 6.5879, 'eval_samples_per_second': 4.402, 'eval_steps_per_second': 0.607, 'epoch': 0.6}
{'loss': 0.3105, 'learning_rate': 0.0005, 'epoch': 0.67}
{'eval_loss': 0.21604782342910767, 'eval_runtime': 6.5411, 'eval_samples_per_second': 4.433, 'eval_steps_per_second': 0.612, 'epoch': 0.67}
{'loss': 0.4153, 'learning_rate': 0.0005, 'epoch': 0.75}
{'eval_loss': 0.2053249180316925, 'eval_runtime': 6.6062, 'eval_samples_per_second': 4.39, 'eval_steps_per_second': 0.605, 'epoch': 0.75}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.4842, 'learning_rate': 0.0005, 'epoch': 0.82}
{'eval_loss': 0.21323342621326447, 'eval_runtime': 6.6161, 'eval_samples_per_second': 4.383, 'eval_steps_per_second': 0.605, 'epoch': 0.82}
{'loss': 0.3226, 'learning_rate': 0.0005, 'epoch': 0.9}
{'eval_loss': 0.194092258810997, 'eval_runtime': 6.6168, 'eval_samples_per_second': 4.383, 'eval_steps_per_second': 0.605, 'epoch': 0.9}
{'loss': 0.3111, 'learning_rate': 0.0005, 'epoch': 0.97}
{'eval_loss': 0.1798645555973053, 'eval_runtime': 6.6049, 'eval_samples_per_second': 4.391, 'eval_steps_per_second': 0.606, 'epoch': 0.97}
{'train_runtime': 296.4646, 'train_samples_per_second': 0.894, 'train_steps_per_second': 0.226, 'train_loss': 0.655930345627799, 'epoch': 1.0}


# Try the model

In [ ]:
# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n\u00bfC\u00f3mo se demuestra el TRL 7? [/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] <<SYS>>
Este es un asistente especializado en el modelo de Niveles de Madurez Tecnológica (TRL) de la NASA, capaz de evaluar y clasificar tecnologías según los 9 niveles de madurez. Debes también responder consultas sobre el TRL, abarcando definiciones, actividades recomendadas y criterios de progresión, actuando como una fuente de conocimiento sobre cómo avanzar tecnologías desde la concepción hasta la comercialización.
<</SYS>>

¿Cómo se demuestra el TRL 7? [/INST] El TRL 7 se demuestra mediante pruebas reales en condiciones operacionales del sistema tecnológico. Esto implica probar el sistema en su entorno real, con los mismos parámetros y condiciones que


#Run Inference
Testeando dos temperaturas diferentes

In [ ]:
from transformers import pipeline

prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n\u00bfC\u00f3mo se demuestra el TRL 8? [/INST]"
num_new_tokens = 100  # change to the number of new tokens you want to generate

# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt, temperature=0.1)
print(result[0]['generated_text'].replace(prompt, ''))

print("----------")
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt, temperature=15)
print(result[0]['generated_text'].replace(prompt, ''))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


KeyboardInterrupt: 

# PREPARACIÓN EVALUACIÓN DE MODELOS LLAMA-2 VS GPT-3.5-TURBO

In [ ]:
import time
import json
import openai
from transformers import pipeline

test_temperature=0.1



def generate_llama2(system_message, prompt):
  num_new_tokens = 100  # change to the number of new tokens you want to generate

  prompt_llama = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{prompt} [/INST]"
  # Count the number of tokens in the prompt
  num_prompt_tokens = len(tokenizer(prompt_llama)['input_ids'])

  # Calculate the maximum length for the generation
  max_length = num_prompt_tokens + num_new_tokens
  #formato del prompt

  inicio_llama_2 = time.time()  # Captura el tiempo de inicio

  #generación de respuesta
  gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
  result = gen(prompt_llama)
  fin_llama_2 = time.time()  # Captura el tiempo de finalización
  tiempo_ejecucion_llama_2 = fin_llama_2 - inicio_llama_2  # Calcula el tiempo de ejecución

  print("Respuesta LLAMA-2:")
  print(result[0]['generated_text'].replace(prompt_llama, ''))
  print(tiempo_ejecucion_llama_2)
  print("-----------------------------------------------------------------------\n\n")
  return tiempo_ejecucion_llama_2, result[0]['generated_text'].replace(prompt_llama, '')

def generate_gpt(system_message, prompt):

 #formato del prompt
  test_messages = []
  test_messages.append({"role": "system", "content": system_message})
  test_messages.append({"role": "user", "content": prompt})

  openai.api_key = "API KEY PERSONAL DE OPENAI"
  fine_tuned_model_id = "ID DEL MODELO CUSTOM EN OPENAI"

  inicio_gpt = time.time()  # Captura el tiempo de inicio

  #Generación de la respuesta
  response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=test_temperature, max_tokens=500
  )
  fin_gpt = time.time()  # Captura el tiempo de finalización
  tiempo_ejecucion_gpt = fin_gpt - inicio_gpt  # Calcula el tiempo de ejecución

  print("Respuesta GPT-3.5-TURBO:")
  print(response["choices"][0]["message"]["content"])
  print(tiempo_ejecucion_gpt)
  print("-----------------------------------------------------------------------\n\n")

  return tiempo_ejecucion_gpt, response["choices"][0]["message"]["content"]

Prueba con la posición 5 del dataset

In [ ]:
print(valid_dataset[14]['prompt'])
print(valid_dataset[14]['response'])

¿Qué tipo de resultados se obtienen en el TRL 5?
Los principales resultados en TRL 5 son el desarrollo y validación de prototipos de la tecnología en condiciones operacionales similares a las reales.


In [ ]:
prompt_test = valid_dataset[14]['prompt']

time_llama, message_llama = generate_llama2(system_message, prompt_test)
time_gpt, message_gpt = generate_gpt(system_message, prompt_test)

Respuesta LLAMA-2:
 Los resultados del TRL 5 son prototipos de prueba en condiciones reales de operación. Se obtienen resultados de calificación y validación de los prototipos. Se demuestra la viabilidad del sistema en condiciones reales. Se obtiene información sobre el rendimiento del sistema en condiciones reales. Se identifica el rendimiento del sistema en condiciones reales. Se obtiene información sobre el rendimiento del sistema en cond
39.09650373458862
-----------------------------------------------------------------------


Respuesta GPT-3.5-TURBO:
Los principales resultados del TRL 5 son la validación de los componentes y la demostración de que el sistema funciona como un todo integrado.
3.7983615398406982
-----------------------------------------------------------------------




Generación del dataset de evaluacion de métricas

In [ ]:
import time
import json
import openai
from transformers import pipeline

evaluation_fase_dataset = []
contador=0

for data in valid_dataset:
  contador += 1

  time_llama, message_llama = generate_llama2(system_message, data['prompt'])
  time_gpt, message_gpt = generate_gpt(system_message, data['prompt'])

  # Almacenar los resultados en la lista
  evaluation_fase_dataset.append({
      'prompt': data['prompt'],
      'referencia': data['response'],
      'generado_llama_2': message_llama,
      'tiempo_ejecucion_llama_2': time_llama,
      'generado_gpt': message_gpt,
      'tiempo_ejecucion_gpt': time_gpt
  })

  print(f'completado {contador} de {len(valid_dataset)}')


print(evaluation_fase_dataset)

file_name = "evaluation_fase_dataset_temp" + str(test_temperature)+ ".jsonl"
with open(file_name, 'w', encoding="utf8") as file:
            for comparacion in evaluation_fase_dataset:
                json_line = json.dumps(comparacion)
                file.write(json_line + '\n')

Respuesta LLAMA-2:
 En el TRL 1 se realizan actividades de investigación básica y experimentación para identificar los conceptos tecnológicos y validar los principios fundamentales. No se realizan actividades de desarrollo tecnológico.

En el TRL 1 se realizan actividades como la identificación de los conceptos tecnológicos, la formulación de hipótesis, la realización de experimentos para validar los principios fundamentales,
39.12927603721619
-----------------------------------------------------------------------


Respuesta GPT-3.5-TURBO:
En el TRL 1 se realizan actividades de investigación básica, formulación de conceptos y publicaciones de artículos científicos. No se realizan pruebas experimentales.
0.6965703964233398
-----------------------------------------------------------------------


completado 1 de 29
Respuesta LLAMA-2:
 En el TRL 7 se realizan pruebas en condiciones reales del sistema, se obtiene la certificación y se demuestra su capacidad de operación en el entorno real